In [ ]:
from mayavi import mlab
mlab.init_notebook()
import sys
sys.path.append("../focusadd")

In [ ]:
import numpy as numpy
import jax.numpy as np
import jax
from jax.ops import index_add
import mayavi as maya
from numpy import cos, sin

In [ ]:
from surface.readAxis import readAxis
from surface.Surface import Surface
from coils.CoilSet import CoilSet
from shapeGradient.ShapeGradient import ShapeGradient

In [ ]:
def getSurface():
    # Initialize the arguments to be used by the program
    

    # Read and return the axis
    axis, epsilon, minor_rad, N_rotate, zeta_off = readAxis("../focusadd/initFiles/axes/ellipticalAxis3Rotate.txt",64)
    # Create the surface
    return Surface(axis, 64, 32, epsilon, minor_rad, N_rotate, zeta_off,1.0)

def getCoilSet():
    surface = getSurface()
    coilSet = CoilSet(surface,input_file="../focusadd/coils/saved/16coilsElliptical3Rotate.hdf5")
    coilSet.set_params(coilSet.get_params())
    return coilSet

def getShapeGradient(surface,coilSet):
    shapegrad = ShapeGradient(surface, coilSet)
    return shapegrad.coil_gradient()

In [ ]:
def plotCoilSet():
    coilset = getCoilSet()
    r_coils = coilset.get_r()
    r_central = coilset.get_r_central()
    surface = getSurface()
    shape_grad = getShapeGradient(surface, coilset)
    shape_grad = jax.ops.index_update(shape_grad, jax.ops.index[:,0,:,:,:], 0.)
    shape_grad = jax.ops.index_update(shape_grad, jax.ops.index[:,-1,:,:,:], 0.)
    r = surface.get_r()
    x = r[:,:,0]
    y = r[:,:,1]
    z = r[:,:,2]
    p = mlab.mesh(x,y,z,colormap="bone")
    for ic in range(r_coils.shape[0]):
        for n in range(r_coils.shape[2]):
            for b in range(r_coils.shape[3]):
                p = mlab.plot3d(r_coils[ic,:,n,b,0],r_coils[ic,:,n,b,1],r_coils[ic,:,n,b,2],tube_radius=0.002, line_width = 0.01, colormap="Spectral")
                p = mlab.quiver3d(r_coils[ic,:,n,b,0],r_coils[ic,:,n,b,1],r_coils[ic,:,n,b,2],shape_grad[ic,:,n,b,0],shape_grad[ic,:,n,b,1],shape_grad[ic,:,n,b,2])
    return p

In [ ]:
mlab.clf()
p = plotCoilSet()
p